# Notebook 1

## Project 5

#### Quang, Paul and Luke

To use the YouTube Analytics API, we have to use the OAuth2 process to access YouTube data from the authenticated user. Our goal with this notebook is to allow any YouTube user (google user) to run this program and create a json file that will contain a token and refresh token that they can use in subsequent notebooks. They can then use this file to query the YouTube API in interesting ways.

To begin the OAuth2 process, we have to create a new project with Google from our resource owner account, which will be a gmail account associated with a YouTube account. After creating the project, we used a py file to store relevant information that we got back from a json file. Our client credentials and redirect uri were then accessed from this file in our notebook. We do this to keep sensitive information private from the notebook, so our work is more accessible.

In [1]:
import importlib
import keys
importlib.reload(keys)
import json
import pprint
keychain = keys.keychain        

# imports our py file that will help build our uri

pp = pprint.PrettyPrinter(indent=2)

client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]

# creating values for our client ID, scope and redirect URI in our json file

print(client_id[:20], scope, redirect)
keychain['youtube']

126920892433-e58m7pn ['https://www.googleapis.com/auth/youtube.readonly'] urn:ietf:wg:oauth:2.0:oob


{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

We then created a session using this information in our file. This gave us an authorization URL that allowed us to gain an access code. 


In [2]:
from requests_oauthlib import OAuth2Session

In [3]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)

# creates our OAuth2 session using the requests library

auth_url, state = session.authorization_url(keychain['youtube']['auth_uri'],access_type='offline',include_granted_scopes='true')

# creates an authorization URL for our user based on their information

print(auth_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=6I7qaOOJ3mt0lZ5BhP3eQrH3J8UcUq&access_type=offline&include_granted_scopes=true


From there, we were able to gain an authorization code to give to the YouTube server. At this point in the Oauth process, we were confident that we had authenticated ourselves and our client to the YouTube servers. 

In [4]:
code = '4/xe4WcQeCYyzXOUmfymrLdnPc6-JEZTNG_PC4x7_Ni8I'

In [5]:
token = session.fetch_token(keychain['youtube']['token_uri'],
                            code=code,
                            client_secret=keychain['youtube']['client_secret'])  

# fetches our access token

InvalidGrantError: (invalid_grant) 

The next step was to allow our client to gain our access token and store it safely. A problem we did not initally anticipate was the expiration of the token we received. After a certain amount of time, the token we saved to use to query the API did not work. We needed a way to continue the program without burdening a user with gaining a new token on their own.

To be able to continue the program for our user, we also needed to retrieve a refresh token that would allow us to continue our access to the API. Once we gained the access token and refresh token, we wrote those tokens to a json file that could be accessed in our subsequent notebook.


In [ ]:
refresh_url = keychain['youtube']['token_uri']
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'])

# fetches our refresh token

token

In [ ]:
import json
with open('token.json', 'w') as outfile:
    json.dump(token, outfile)
    
# takes our original json file and dumps it out to our local directory with updated token

The main challenge of this notebook was making it accessible for any user at any time, without compromising their personal information. Through the use of files outside the program and the generation of a refresh token, we accomplished our goal of completing the Oauth process for the user. The user can now use our subsequent notebooks to explore the YouTube API and access the visuals we created in our exploration.